# TensorRT-LLM: Llama-3-Taiwan-8B-Instruct
TensorRT-LLM provides users with an easy-to-use Python API to define Large Language Models (LLMs) and build TensorRT engines that contain state-of-the-art optimizations to perform inference efficiently on NVIDIA GPUs. TensorRT-LLM also contains components to create Python and C++ runtimes that execute those TensorRT engines. It also includes a backend for integration with the NVIDIA Triton Inference Server. Models built with TensorRT-LLM can be executed on a wide range of configurations going from a single GPU to multiple nodes with multiple GPUs (using Tensor Parallelism).

In [ ]:
! pip install datasets==2.19 rouge_score
! git clone https://github.com/triton-inference-server/tensorrtllm_backend.git -b v0.9.0 --single-branch
%cd tensorrtllm_backend/
! git lfs install
! git submodule update --init --recursive
%cd /workspace

### Download model from Huggingface

In [ ]:
!huggingface-cli download yentinglin/Llama-3-Taiwan-8B-Instruct --local-dir Llama-3-Taiwan-8B-Instruct --local-dir-use-symlinks=False 

### 1. Build TensorRT-LLM engines - FP16

**TensorRT-LLM** builds TensorRT engine(s) from HF checkpoint. If no checkpoint directory is specified, TensorRT-LLM will build engine(s) with dummy weights.

Normally trtllm-build only requires single GPU, but if you've already got all the GPUs needed for inference, you could enable parallel building to make the engine building process faster by adding --workers argument. Please note that currently workers feature only supports single node.

In [ ]:
# Define model weight path, output checkpoint path and output engine path
%env HF_LLAMA_MODEL=/workspace/Llama-3-Taiwan-8B-Instruct
%env UNIFIED_CKPT_PATH=ckpt/llama/8b/fp16
%env ENGINE_PATH=engines/llama/8b/fp16

!python tensorrtllm_backend/tensorrt_llm/examples/llama/convert_checkpoint.py --model_dir ${HF_LLAMA_MODEL} \
--output_dir ${UNIFIED_CKPT_PATH} \
--dtype float16

!trtllm-build --checkpoint_dir ${UNIFIED_CKPT_PATH} \
             --remove_input_padding enable \
             --gpt_attention_plugin float16 \
             --context_fmha enable \
             --gemm_plugin float16 \
             --output_dir ${ENGINE_PATH} \
             --paged_kv_cache enable \
             --max_batch_size 64

### Run FP16 engine Inference
To run a TensorRT-LLM LLaMA model using the engines generated by trtllm-build

In [ ]:
!python3 tensorrtllm_backend/tensorrt_llm/examples/run.py \
--max_output_len 50 \
--tokenizer_dir ${HF_LLAMA_MODEL} \
--engine_dir ${ENGINE_PATH}

### 2. Build TensorRT-LLM engines - INT8 KV cache + per-channel weight-only quantization
To maximize performance and reduce memory footprint, TensorRT-LLM allows the models to be executed using different quantization modes. TensorRT-LLM supports INT4 or INT8 weights (and FP16 activations; a.k.a. INT4/INT8 weight-only) as well as a complete implementation of the SmoothQuant technique.

In [ ]:
# Define model weight path, output checkpoint path and output engine path
%env HF_LLAMA_MODEL=/workspace/Llama-3-Taiwan-8B-Instruct
%env UNIFIED_CKPT_PATH=ckpt/llama/8b/int8
%env ENGINE_PATH=engines/llama/8b/int8

!python tensorrtllm_backend/tensorrt_llm/examples/llama/convert_checkpoint.py --model_dir ${HF_LLAMA_MODEL} \
--output_dir ${UNIFIED_CKPT_PATH} \
--dtype float16 \
--int8_kv_cache \
--use_weight_only \
--weight_only_precision int8

!trtllm-build --checkpoint_dir ${UNIFIED_CKPT_PATH} \
             --remove_input_padding enable \
             --gpt_attention_plugin float16 \
             --context_fmha enable \
             --gemm_plugin float16 \
             --output_dir ${ENGINE_PATH} \
             --paged_kv_cache enable \
             --max_batch_size 64

### Summarization using the LLaMA model

In [ ]:
!python tensorrtllm_backend/tensorrt_llm/examples/summarize.py \
--data_type fp16 \
--test_hf \
--hf_model_dir ${HF_LLAMA_MODEL} \
--test_trt_llm \
--engine_dir ${ENGINE_PATH} \
--max_ite 10

### 3. Build TensorRT-LLM engines - FP8 Post-Training Quantization

The examples below uses the NVIDIA Modelopt (AlgorithMic Model Optimization) toolkit for the model quantization process.
Although the V100 does not support the FP8 datatype, we have included it as a reference.

In [ ]:
# Define model weight path, output checkpoint path and output engine path
# %env HF_LLAMA_MODEL=/workspace/Llama-3-Taiwan-8B-Instruct
# %env UNIFIED_CKPT_PATH=ckpt/llama/8b/fp8
# %env ENGINE_PATH=engines/llama/8b/fp8

# !python tensorrtllm_backend/tensorrt_llm/examples/quantization/quantize.py --model_dir ${HF_LLAMA_MODEL} \
# --dtype float16 \
# --qformat fp8 \
# --kv_cache_dtype fp8 \
# --output_dir ${UNIFIED_CKPT_PATH} \
# --calib_size 512


# !trtllm-build --checkpoint_dir ${UNIFIED_CKPT_PATH} \
#              --remove_input_padding enable \
#              --gpt_attention_plugin float16 \
#              --context_fmha enable \
#              --gemm_plugin float16 \
#              --output_dir ${ENGINE_PATH} \
#              --paged_kv_cache enable \
#              --max_batch_size 64

### 4. Build TensorRT-LLM engines - Groupwise quantization (AWQ/GPTQ)
One can enable AWQ/GPTQ INT4 weight only quantization with these options when building engine with trtllm-build:
NVIDIA Modelopt toolkit is used for AWQ weight quantization. Please see [examples/quantization/README.md](tensorrtllm_backend/tensorrt_llm/examples/quantization/README.md) for Modelopt installation instructions.

In [ ]:
# Define model weight path, output checkpoint path and output engine path
%env HF_LLAMA_MODEL=/workspace/Llama-3-Taiwan-8B-Instruct
%env UNIFIED_CKPT_PATH=ckpt/llama/8b/int4
%env ENGINE_PATH=engines/llama/8b/int4

# Quantize HF LLaMA 8B checkpoint into INT4 AWQ format
!python tensorrtllm_backend/tensorrt_llm/examples/quantization/quantize.py \
--model_dir ${HF_LLAMA_MODEL} \
--dtype float16 \
--qformat int4_awq \
--awq_block_size 128 \
--output_dir ${UNIFIED_CKPT_PATH} \
--calib_size 16

!trtllm-build --checkpoint_dir ${UNIFIED_CKPT_PATH} \
             --remove_input_padding enable \
             --gpt_attention_plugin float16 \
             --context_fmha enable \
             --gemm_plugin float16 \
             --output_dir ${ENGINE_PATH} \
             --paged_kv_cache enable \
             --max_batch_size 64

# Triton Inference Server with TensorRT-LLM backend: Llama-3-8B-Instruct Deployment using Triton Inference Server

The Triton for [TensorRT-LLM](https://github.com/NVIDIA/TensorRT-LLM) backend. You can learn more about Triton backends in the [backend repo](https://github.com/triton-inference-server/tensorrtllm_backend/tree/main). The goal of TensorRT-LLM Backend is to let you serve TensorRT-LLM models with Triton Inference Server.

## Using the TensorRT-LLM Backend
We will look at 4 steps to serve the TensorRT-LLM model with the Triton TensorRT-LLM Backend on a 1-GPU environment. The example uses [meta-llama/Meta-Llama-3-8B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct) from the TensorRT-LLM repository.

### 1. Build TensorRT-LLM engines

### 2. Prepare inference configs

There are four models in the all_models/inflight_batcher_llm directory that will be used in this example: preprocessing -> tensorrt_llm -> postprocessing

- **preprocessing**: This model is used for tokenizing, meaning the conversion from prompts(string) to input_ids(list of ints).
- **tensorrt_llm**: This model is a wrapper of your TensorRT-LLM model and is used for inferencing
- **postprocessing**: This model is used for de-tokenizing, meaning the conversion from output_ids(list of ints) to outputs(string).
- **ensemble**: This model is used to chain the three models above together.

In [ ]:
%env HF_LLAMA_MODEL=/workspace/Llama-3-Taiwan-8B-Instruct
%env ENGINE_PATH=engines/llama/8b/fp16

In [ ]:
%env BS=64
!rm -rf tensorrtllm_backend/llama_ifb
!cp -r tensorrtllm_backend/all_models/inflight_batcher_llm/ tensorrtllm_backend/llama_ifb

!python3 tensorrtllm_backend/tools/fill_template.py -i tensorrtllm_backend/llama_ifb/preprocessing/config.pbtxt tokenizer_dir:${HF_LLAMA_MODEL},triton_max_batch_size:$BS,preprocessing_instance_count:1
!python3 tensorrtllm_backend/tools/fill_template.py -i tensorrtllm_backend/llama_ifb/postprocessing/config.pbtxt tokenizer_dir:${HF_LLAMA_MODEL},triton_max_batch_size:$BS,postprocessing_instance_count:1
!python3 tensorrtllm_backend/tools/fill_template.py -i tensorrtllm_backend/llama_ifb/tensorrt_llm_bls/config.pbtxt triton_max_batch_size:$BS,decoupled_mode:False,bls_instance_count:1,accumulate_tokens:False
!python3 tensorrtllm_backend/tools/fill_template.py -i tensorrtllm_backend/llama_ifb/ensemble/config.pbtxt triton_max_batch_size:$BS
!python3 tensorrtllm_backend/tools/fill_template.py -i tensorrtllm_backend/llama_ifb/tensorrt_llm/config.pbtxt triton_backend:tensorrtllm,triton_max_batch_size:$BS,decoupled_mode:False,max_beam_width:1,engine_dir:${ENGINE_PATH},max_tokens_in_paged_kv_cache:2560,max_attention_window_size:2560,kv_cache_free_gpu_mem_fraction:0.5,exclude_input_in_output:True,enable_kv_cache_reuse:False,batching_strategy:inflight_fused_batching,max_queue_delay_microseconds:0

### 3. Launch Triton server
Open a terminal and run the following code:
```bash
cd /workspace/
python /workspace/tensorrtllm_backend/scripts/launch_triton_server.py --world_size 1 --model_repo=/workspace/tensorrtllm_backend/llama_ifb/
```

## Query the server with the Triton-generated endpoint
You can query the server using Triton's generate endpoint with a curl command based on the following general format within your client environment/container:

In [ ]:
!curl -X POST localhost:8000/v2/models/ensemble/generate -d \
'{"text_input": "What is machine learning?", \
"max_tokens": 20, \
"bad_words": "", \
"stop_words": "", \
"pad_id": 2, \
"end_id": 2}'

## Querying and Formatting using Python
We notice the format is not quite useful, let us now try to do the same via Python, here is a snippet in Python that does the same as above, let us run it now:

In [ ]:
import requests
import json
import os

# Retrieve the HTTP port from environment variables
http_port = 8000

# Check if HTTP_PORT is set
if http_port is None:
    print("Error: HTTP_PORT environment variable is not set.")
    exit(1)

# Set the URL with the HTTP port
url = f'http://localhost:{http_port}/v2/models/ensemble/generate'

In [ ]:
# Define the payload
input_text = "What is machine learning?"
payload = {
    "text_input": input_text,
    "max_tokens": 1024,
    "bad_words": "",
    "stop_words": "<|eot_id|>"
}

# Make a POST request
response = requests.post(url, json=payload)

# Check if the request was successful
if response.status_code == 200:
    # Parse the response
    data = response.json()
    output_text = data.get('text_output')

    # Format and print the output
    print(f"Input: {input_text}")
    print(f"Output: {output_text}")
else:
    print(f"Error: {response.status_code}")

In [ ]:
# Define the payload
input_text = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\
You are a helpful AI assistant<|eot_id|><|start_header_id|>user<|end_header_id|>\n\
What is machine learning?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"

payload = {
    "text_input": input_text,
    "max_tokens": 1024,
    "bad_words": "",
    "stop_words": "<|eot_id|>"
}

# Make a POST request
response = requests.post(url, json=payload)

# Check if the request was successful
if response.status_code == 200:
    # Parse the response
    data = response.json()
    output_text = data.get('text_output')

    # Format and print the output
    print(f"Input: {input_text}")
    print(f"Output: {output_text}")
else:
    print(f"Error: {response.status_code}")

## Kill the server

In [ ]:
!pgrep mpirun | xargs kill